## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
# All codes modified to python 2
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [7]:
import urllib2
import zipfile
import lxml.etree

In [11]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    f = urllib2.urlopen("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip")

In [12]:
data = f.read()
with open("ted_en-20160408.zip", "wb") as code:
    code.write(data)
    
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [13]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

u' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [14]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [15]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

u"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [16]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

[u"Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 u'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 u' Both are necessary, but it can be too much of a good thing',
 u'Consider Facit',
 u" I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [17]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [18]:
len(sentences_ted)

266694

In [19]:
print(sentences_ted[0])
print(sentences_ted[1])

[u'here', u'are', u'two', u'reasons', u'companies', u'fail', u'they', u'only', u'do', u'more', u'of', u'the', u'same', u'or', u'they', u'only', u'do', u'what', u's', u'new']
[u'to', u'me', u'the', u'real', u'real', u'solution', u'to', u'quality', u'growth', u'is', u'figuring', u'out', u'the', u'balance', u'between', u'two', u'activities', u'exploration', u'and', u'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [26]:
import collections
word_dict = collections.Counter()
for sent in sentences_ted:
    for word in sent:
        word_dict[word] += 1
sorted_counts = sorted(word_dict.items(), key=lambda x:x[1], reverse=True)[:1000]
counts_ted_top1000 = [x[1] for x in sorted_counts]

print sorted_counts[:40]

[(u'the', 207748), (u'and', 149305), (u'to', 125169), (u'of', 114818), (u'a', 105399), (u'that', 95146), (u'i', 83180), (u'in', 78070), (u'it', 74738), (u'you', 70923), (u'we', 67629), (u'is', 63251), (u's', 57156), (u'this', 49241), (u'so', 37014), (u'they', 33102), (u'was', 30806), (u'for', 29713), (u'are', 27995), (u'have', 27344), (u'but', 26732), (u'what', 26519), (u'on', 25962), (u'with', 24706), (u'can', 23377), (u't', 22757), (u'about', 21246), (u'there', 21041), (u'be', 20201), (u'as', 19488), (u'at', 19216), (u'all', 19021), (u'not', 18626), (u'do', 17928), (u'my', 17908), (u'one', 17551), (u're', 17012), (u'people', 16723), (u'like', 16273), (u'if', 15868)]


Plot distribution of top-1000 words

In [23]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [28]:
from gensim.models import Word2Vec

In [33]:
# min_count: minimum number of word counts for pruning the internal dictionary, to ignore infrequent words
# size: size of the NN layer
model_ted = Word2Vec(sentences_ted, min_count=10, size=100) # Train the word embedding

In [36]:
# size of ted vocabulary
len(model_ted.vocab)

14427

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [48]:
model_ted.most_similar("man")
# or model_ted.most_similar(positive=['man'], topn=10)

[(u'woman', 0.7562875747680664),
 (u'guy', 0.7457346320152283),
 (u'boy', 0.6716436743736267),
 (u'kid', 0.6608802080154419),
 (u'girl', 0.6252491474151611),
 (u'lady', 0.6212112903594971),
 (u'curator', 0.5707521438598633),
 (u'gentleman', 0.5680621862411499),
 (u'widow', 0.5636664628982544),
 (u'soldier', 0.5577048659324646)]

In [49]:
model_ted.most_similar("computer")

[(u'machine', 0.6358190774917603),
 (u'graphics', 0.5718486905097961),
 (u'game', 0.5625736713409424),
 (u'video', 0.5560340881347656),
 (u'software', 0.5558165311813354),
 (u'keyboard', 0.5425848364830017),
 (u'circuit', 0.5348063707351685),
 (u'laser', 0.5312381386756897),
 (u'mechanical', 0.5312161445617676),
 (u'portable', 0.5286329388618469)]

In [50]:
# Check the cosine similarity of two words
print "Computed by gensim: ", model_ted.similarity('woman', 'man')
cos_sim = np.dot(model_ted['woman'], model_ted['man']) / (np.linalg.norm(model_ted['woman']) * np.linalg.norm(model_ted['man']))
print "Computed by hand: ", cos_sim

Computed by gensim:  0.75628761317
Computed by hand:  0.756288


#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [56]:
# This assumes words_top_ted is a list of strings, the top 1000 words
words_top_ted = [x[0] for x in sorted_counts]
words_top_vec_ted = np.zeros((1000, model_ted.layer1_sizse))
for i, word in enumerate(words_top_ted):
    words_top_vec_ted[i, :] = model_ted[word]
# words_top_vec_ted = model_ted[words_top_ted]

In [57]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [58]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [59]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    f = urllib2.urlopen("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip")
data = f.read()
with open("wikitext-103-raw-v1.zip", "wb") as code:
    code.write(data)

In [86]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = unicode(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), 'utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [83]:
sentences_wiki = []
# remove short sentences
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub("[^a-z]", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i]).split(' ').strip()
    sentences_wiki[s_i] = [w for w in sentences_wiki if w != ''] # remove empty string
del input_text

In [84]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [88]:
# get the top 1000 frequent word
wiki_word_counter = collections.Counter()
for sent in sentences_wiki:
    for word in sent:
        wiki_word_counter[word] += 1
sorted_words = sorted(wiki_word_counter.items(), key=lambda x: x[1], reverse=True)
words_top_wiki = [w[0] for w in sorted_words][:1000] 

model_wiki = Word2Vec(sentences_wiki, min_count=10)

#### t-SNE visualization

In [96]:
print len(model_wiki.vocab)
print model_wiki.layer1_size

48678
100
100


In [97]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = np.zeros((1000, model_wiki.layer1_size))
for i, word in enumerate(words_top_wiki):
    words_top_vec_wiki[i, :] = model_wiki[word]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [ ]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)